In [41]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy as sp

#### Question 1: Solving a Non-Linear Equation

A common problem in numerical analysis is finding the inverse of a Cumulative Distribution Function (CDF). A CDF is a function F(x) that is non-decreasing over some domain [a, b], and for which F(a) = 0 and F(b) = 1. 

#### Definining a function to solve this

In [10]:
# The following function uses the bisectional method to find the root. 
def bisectFunc(f, a, b, tol):
    
    lower, upper = a, b
    
    while upper - lower > tol:
        middle = 0.5 * (upper + lower)
        if f(upper)>f(lower):
            # === if root is between lower and middle === #
            if f(middle) > 0:  
                lower, upper = lower, middle
            # === if root is between middle and upper  === #
            else:              
                lower, upper = middle, upper
        else:
            if f(middle) > 0:  
                lower, upper = middle, upper
            else:              
                lower, upper = lower, middle

    return 0.5 * (upper + lower)

(a) Use the CDF of the uniform distribution defined on [0, 2]. Compute the percentiles associated with the following probabilities: 0.6, 0.4. Comment and report on the accuracy of your results by comparing them with the exact percentile values.

The fuction we are going to use to solve this problem is

$$f = \frac{x-a}{b-a} - p $$

where p is the probability, a is the lower bound and b is the upper bound

In [13]:
# starting with p = 0.4

f = lambda x: (x-0)/(2-0) - 0.4

ans1 = round(bisectFunc(f,0,2,10e-10),3)
ans1

0.8

In [14]:
# next using p = 0.6

f = lambda x: (x-0)/(2-0) - 0.6

ans2 = round(bisectFunc(f,0,2,10e-10),3)
ans2

1.2

To compare with the exact results we just have to use the followinng formula
$$F^{-1}(p)= a + p(b-a)$$

where p is the probability, a is lower bound, and b is upper bound

In [5]:
# for p = 0.4
F_p = 0 + 0.4*(2-0)
F_p

0.8

In [6]:
# for p = 0.6
F_p = 0 + 0.6*(2-0)
F_p

1.2

When comparing my results, we can see that they are identical to the true value

(b) Use the CDF of the negative exponential distribution, with parameter λ = 2. Compute the percentiles associated with the following probabilities: 0.7, 0.3. Comment and report on the accuracy of your results by comparing them with the exact percentile values.

In this part I will be working with a different function, this function will be defined as:
$$f = 1-e^{-\lambda x} - p$$

where lambda = 2, and p is the associated probability

In [31]:
import math

# for p = 0.7
f1 = lambda x: 1-math.exp(-2*x) - 0.7

round(bisectFunc(f1,0,2,10e-10),4)


0.602

In [34]:
# for p = 0.3
f2 = lambda x: 1-math.exp(-2*x) - 0.3

ans2 = round(bisectFunc(f2,0,2,10e-10),4)
ans2

0.1783

In [30]:
from scipy.optimize import fsolve

# we can use fsolve to help us determine the exact percentile value
# for p = 0.7
x0 = 1
round(fsolve(f1,x0)[0],4)

0.602

In [35]:
# for p = 0.3
x0 = 1
round(fsolve(f2,x0)[0],4)

0.1783

We can again see that when comparing my results to exact values they are the same

(c)Use the CDF of the standard normal distribution, with parameters µ = 0 and σ
2 = 1. Compute the percentiles associated with the following probabilities: 0.9, 0.2. 

In [36]:
# To help me solve this question I will be using a scipy library that has a built-in
# feature that that mimics the the CDF of a normal distribution

from scipy.stats import norm

In [38]:
# for p = 0.9
f = lambda x: norm.cdf(x) - 0.2

round(bisectFunc(f,-4,4,10e-10),4)

-0.8416

In [39]:
# for p = 0.2

f = lambda x: norm.cdf(x) - 0.9

round(bisectFunc(f,-4,4,10e-10),4)

1.2816

The numbers that are returned in the following equations are z-scores, and we can look up a z-score table for the standard normal distribution to confirm my answers. 
For instance, when we look at a z-score of 1.28 we get a probability of 0.8997, and for a z-score of -0.84 we get a probability of 0.2005. This confirms that our answers above are correct.
Here is the link for the table I used for reference.
https://cnx.org/contents/SCdvoXS4@2/Using-the-Normal-Distribution

#### Question 2: A Three-period Model of Storage

Consider the market for potatoes, which are storable intra-seasonally (but not inter-seasonally). In this market, the harvest is entirely consumed over three periods, t = 1, 2, 3. Potatoes can be stored at (marginal) cost κ per potato/period. Denoting initial supply by s and consumption in period t by ct, the market
equilibrium requires that:
$$s = c1 + c2 + c3$$
Competition among storers, who have perfect foresight about future prices, eliminates inter-period arbitrage opportunities. Thus,
$$p1 + k = \delta p2$$ and
$$p2 + k = \delta p3 $$

where pt is the equilibrium price in period t, κ = 0.1 is per-period unit cost of storage, and δ = 0.9 is
per-period discount factor. The demand function, which is assumed to be the same across periods, is given
by pt = c
−4

(a) Write a Python code to compute the equilibrium prices for periods 1, 2 and 3 when s = 1, s = 2, and
s = 3. Report the values and comment.

In [52]:
# I am going to be using fsolve to help me solve the system of non-linear equations.
from scipy.optimize import fsolve

In [53]:
# Basically what this function does is set up the system of non-linear equations in a 
# format that fsolve can understand. I define and pass in the three equations we 
# have listed in the problem above. I also add in any parameters that are need for the 
# specific problem
# k = 0.1 , s = 1, delta = 0.9

def myFunc(x):
    c1 = x[0]
    c2 = x[1]
    c3 = x[2]
    
    F = np.empty((3))
    F[0] = c1+c2+c3-1
    F[1] = 0.9*(1/(c2**4))-0.1-(1/(c1**4))
    F[2] = 0.9*(1/(c3**4))-0.1-(1/(c2**4))
    
    return F

In [56]:
# Making an inital guess
xGuess = np.array([2,2,2])

# Calling fsolve on the system and passing in the initial guess
x = fsolve(myFunc, xGuess)
x

array([0.34226229, 0.33325049, 0.32448722])

In [57]:
p1 = 1/(x[0]**4)
p2 = 1/(x[1]**4)
p3 = 1/(x[2]**4)
print("k = 0.1 , s = 1, delta = 0.9")
print("----------------------------")
print("         Eq Values")
print("----------------------------")
print("c1 =", x[0])
print("c2 =", x[1])
print("c3 =", x[2])
print()
print("p1 =", p1)
print("p2 =", p2)
print("p3 =", p3)

k = 0.1 , s = 1, delta = 0.9
----------------------------
         Eq Values
----------------------------
c1 = 0.34226229300147265
c2 = 0.333250487897177
c3 = 0.32448721910135037

p1 = 72.87251823988328
p2 = 81.08057582276557
p3 = 90.20063980278651


I tried to write and pass in different parameters according to the question, however
for some reason I kept getting errors, therefore I had to hardcode, which takes more time and is not as neat but gets you the same answers

In [62]:
# k = 0.1 , s = 2, delta = 0.9

def myFunc(x):
    c1 = x[0]
    c2 = x[1]
    c3 = x[2]
    
    F = np.empty((3))
    F[0] = c1+c2+c3-2
    F[1] = 0.9*(1/(c2**4))-0.1-(1/(c1**4))
    F[2] = 0.9*(1/(c3**4))-0.1-(1/(c2**4))
    
    return F

In [63]:
# Making an inital guess
xGuess = np.array([1,1,1])

# Calling fsolve on the system and passing in the initial guess
x = fsolve(myFunc, xGuess)
x

array([0.68788076, 0.66629975, 0.64581949])

In [64]:
p1 = 1/(x[0]**4)
p2 = 1/(x[1]**4)
p3 = 1/(x[2]**4)
print("k = 0.1 , s = 2, delta = 0.9")
print("----------------------------")
print("         Eq Values")
print("----------------------------")
print("c1 =", x[0])
print("c2 =", x[1])
print("c3 =", x[2])
print()
print("p1 =", p1)
print("p2 =", p2)
print("p3 =", p3)

k = 0.1 , s = 2, delta = 0.9
----------------------------
         Eq Values
----------------------------
c1 = 0.6878807611692905
c2 = 0.6662997497516732
c3 = 0.6458194890790363

p1 = 4.466294407861632
p2 = 5.073660453162372
p3 = 5.748511614610137


In [65]:
# k = 0.1 , s = 3, delta = 0.9

def myFunc(x):
    c1 = x[0]
    c2 = x[1]
    c3 = x[2]
    
    F = np.empty((3))
    F[0] = c1+c2+c3-3
    F[1] = 0.9*(1/(c2**4))-0.1-(1/(c1**4))
    F[2] = 0.9*(1/(c3**4))-0.1-(1/(c2**4))
    
    return F

In [66]:
# Making an inital guess
xGuess = np.array([1,1,1])

# Calling fsolve on the system and passing in the initial guess
x = fsolve(myFunc, xGuess)
x

array([1.05408586, 0.99723572, 0.94867842])

In [67]:
p1 = 1/(x[0]**4)
p2 = 1/(x[1]**4)
p3 = 1/(x[2]**4)
print("k = 0.1 , s = 2, delta = 0.9")
print("----------------------------")
print("         Eq Values")
print("----------------------------")
print("c1 =", x[0])
print("c2 =", x[1])
print("c3 =", x[2])
print()
print("p1 =", p1)
print("p2 =", p2)
print("p3 =", p3)

k = 0.1 , s = 2, delta = 0.9
----------------------------
         Eq Values
----------------------------
c1 = 1.0540858619234086
c2 = 0.9972357181195423
c3 = 0.948678419957049

p1 = 0.8100205681123656
p2 = 1.0111339645692772
p3 = 1.2345932939659066


We can see that doubling and tripling s (the supply) from 1 to 2 and then from 2 to 3, dramtatically effects the prices of the good in each period. Logically this makes sense as if you were to double the supply of something in the market it becomes less valuable and you will see a shift downward in prices. Its also worth nothing that consumption is always the greatest in the first period and then decreasing slowly afterwards, and price is always less in the first period and then slowly increases. This is resulting from the discount factor of 0.9. On average the following periods prices will be atleast 1/0.9 = 1.1 times larger than the previous period. Also because there is a cost of storage of k=0.1 to store the goods per period, this means that the there is a higher cost in holding onto the goods, which results in the lower consumption by the third period

(b) Now solve the model when κ = 0.2 and s = 1. Report the equilibrium values and comment.

In [68]:
# k = 0.2 , s = 1, delta = 0.9
def myFunc(x):
    c1 = x[0]
    c2 = x[1]
    c3 = x[2]
    
    F = np.empty((3))
    F[0] = c1+c2+c3-1
    F[1] = 0.9*(1/(c2**4))-0.2-(1/(c1**4))
    F[2] = 0.9*(1/(c3**4))-0.2-(1/(c2**4))
    
    return F

In [69]:
xGuess = np.array([1,1,1])
x = fsolve(myFunc, xGuess)
x

array([0.34237375, 0.3332446 , 0.32438165])

In [71]:
p1 = 1/(x[0]**4)
p2 = 1/(x[1]**4)
p3 = 1/(x[2]**4)
print("k = 0.2 , s = 1, delta = 0.9")
print("----------------------------")
print("         Eq Values")
print("----------------------------")
print("c1 =", x[0])
print("c2 =", x[1])
print("c3 =", x[2])
print()
print("p1 =", p1)
print("p2 =", p2)
print("p3 =", p3)

k = 0.2 , s = 1, delta = 0.9
----------------------------
         Eq Values
----------------------------
c1 = 0.3423737470774766
c2 = 0.3332446008953794
c3 = 0.324381652027144

p1 = 72.77767482185594
p2 = 81.08630535768158
p3 = 90.31811706408006


The above example represents an increase in the per-period unit cost of storage. When comparing this to the very first example where we had s = 1, delta = 0.9, and k = 0.1,
we can see that the prices and consumption levels in each period are very similar. 
We have decreasing consumption as the periods progress and increasing prices.  

(c) Now solve the model when κ = 0.1, s = 1 and δ = 0.5 . Report the equilibrium values and comment.

In [78]:
# k = 0.1 , s = 1, delta = 0.5
def myFunc(x):
    c1 = x[0]
    c2 = x[1]
    c3 = x[2]
    
    F = np.empty((3))
    F[0] = c1+c2+c3-1
    F[1] = 0.5*(1/(c2**4))-0.1-(1/(c1**4))
    F[2] = 0.5*(1/(c3**4))-0.1-(1/(c2**4))
    
    return F

In [79]:
xGuess = np.array([1,1,1])
x = fsolve(myFunc, xGuess)
x

array([0.39263796, 0.32997197, 0.27739007])

In [80]:
p1 = 1/(x[0]**4)
p2 = 1/(x[1]**4)
p3 = 1/(x[2]**4)
print("k = 0.1 , s = 1, delta = 0.5")
print("----------------------------")
print("         Eq Values")
print("----------------------------")
print("c1 =", x[0])
print("c2 =", x[1])
print("c3 =", x[2])
print()
print("p1 =", p1)
print("p2 =", p2)
print("p3 =", p3)

k = 0.1 , s = 1, delta = 0.5
----------------------------
         Eq Values
----------------------------
c1 = 0.3926379606616476
c2 = 0.3299719693723708
c3 = 0.27739006996598164

p1 = 42.075652397751554
p2 = 84.35130479567195
p3 = 168.90260958921357


This example is where we get the largest disparity between price levels as time progress. This is all a result of decreasing the discount factor from 0.9 to 0.5. Because the discount factor is 0.5, you can expect to see the next periods prices being
1/0.5 = 2 times as big as the previous periods, which is exactly what we see. With such  a large increase is prices period over period you would expect consumption to fall period over period, which is also exaclty what we see in the model. In all other instances the consumption values period over period where relatively close, but in this example because of the large increases in price we can see that consumption is the most disperse.

In [81]:
!jupyter-nbconvert --to PDFviaHTML Assignment_3.pdf

[NbConvertApp] WARNING | pattern 'Assignment_3.pdf' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--debug
    set log level to logging.DEBUG (maximize logging output)
--generate-config
    generate default config file
-y
    Answer yes to any questions instead of prompting.
--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--stdout
    Write notebook output to stdout